# Set constant

In [1]:
max_len = 20 # max sentence size

# Load libaries

In [2]:
from freq_utils import *

import regex as re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score

from tensorflow.keras.utils import to_categorical

pd.options.display.max_colwidth = 200

# Load dataset

In [3]:
df0 = pd.read_csv('data/TrueOrganized.csv')
df1 = pd.read_csv('data/FakeOrganized.csv')
df0['label'] = 0
df1['label'] = 1

# Make dictionaries

### Pretrained word embeddings
- Word to index
- Word to vector

In [4]:
word_to_index, index_to_word, word_to_vector = get_pretrained_embedding()

### PoS tag encodings
- PoS word to index

In [5]:
df = pd.concat([df0.cleaned_pos, df1.cleaned_pos])

pos_set = set()
for x in list(df.str.lower().str.split()):
    pos_set.update(x)

pos_list = list(pos_set)
pos_to_index = { pos_list[i]: i for i in range(len(pos_list)) }

print(pos_to_index)

{'vb': 0, 'rp': 1, 'wrb': 2, 'to': 3, 'fw': 4, 'wp$': 5, 'ex': 6, 'rb': 7, 'jjs': 8, "''": 9, 'prp$': 10, 'pos': 11, 'pdt': 12, 'nns': 13, 'vbz': 14, '$': 15, 'uh': 16, 'nnp': 17, 'wp': 18, 'jjr': 19, 'rbr': 20, 'prp': 21, 'in': 22, 'vbn': 23, '.': 24, 'vbd': 25, 'cc': 26, 'md': 27, 'vbp': 28, 'wdt': 29, ':': 30, 'nn': 31, 'cd': 32, 'jj': 33, 'vbg': 34, 'rbs': 35, 'dt': 36, 'sym': 37, 'nnps': 38}


In [6]:
len(pos_to_index)

39

# Train/dev/test split

In [7]:
train, dev, test = train_dev_test_split([df0, df1], m=40000, class_column='label', 
                                    class_balance=True, r_dev=0.2, r_test=0.2, rand_state=42)

# Define models

In [8]:
def simple_Manual(test):
    
    y_true = test.label
    y_pred = []
    
    # title size > 20?
    y_pred.append( test.apply(lambda row: 1 if len(row['lower_title'].split())> 20 else 0, axis=1) )
    # noise > 3 
    y_pred.append( test.apply(lambda row: 1 if len(re.findall(re.compile('[^\s\w]'), row['lower_title'])) > 5 else 0, axis=1) )
    # clickbait, slang, first names
    trigger_word = ['_mytag_parentheses_', '_mytag_slang_',
                    'donald','obama','hillary','bernie']

    for i in range(len(trigger_word)):
        y_pred.append( test.minimal_words.str.contains(trigger_word[i])*1 )

    cut_name = ['too_long','noisy','clickbait','slang'] + trigger_word[-4:]

    return cut_name, y_true, y_pred

In [9]:
def simple_NB(train,dev,test,Xname='title',Yname='label'):
    
    train = pd.concat([train,dev])
    
    X_train = train[Xname].tolist()
    Y_train = train[Yname].tolist()

    X_test = test[Xname].tolist()
    Y_test = test[Yname]#.tolist() -> commented out to keep dataframe index
    
    counter = CountVectorizer()

    counter.fit(X_train+X_test)

    train_counts = counter.transform(X_train)
    test_counts = counter.transform(X_test)

    #print(counter.vocabulary_)

    classifier = MultinomialNB()
    classifier.fit(train_counts,Y_train)
    
    predict = classifier.predict(test_counts)
    
    proba = classifier.predict_proba(test_counts)
    
    model_name = 'Naive Bayse - '+Xname
    
    
    return model_name, predict, Y_test, proba, classifier, counter

In [10]:
def model_FNN(input_shape, word_to_index, word_to_vector, n_class=2, trainable=False):
    '''
    input_shape: (max_len,)
    word_to_index: word to index dictionary
    word_to_vector: word to embedding vector dictionary

    return model

    then
    X: Indices of a sentence (m, max_len)
    Y: Class probability, one hot vector (m, # classes)
    '''

    # Input layer
    # X_indices (m, max_len)
    X_input = tfl.Input(shape=input_shape, dtype='int32')
    
    X = X_input
    if bool(word_to_vector):
        # Word embeding
        # Output: (m, max_len, emb_dim)
        # Embedding layer
        embedding_layer = pretrained_embedding_layer(word_to_vector, word_to_index, trainable=trainable)
        X = embedding_layer(X)      
    else:
        # One hot encoding (=input)
        # Output: (m, max_len, num_cat)
        X = tf.dtypes.cast(X, tf.float32)    

    # Masking layer
    # skip zero vector words
    X = tfl.Masking(mask_value=0.)(X)
    
    # Take average
    max_len = X.shape[1]
    # Make a list from slice
    X_avg = [ X[:,i,:] for i in range(max_len) ]
    # Take average of a sentence
    X = tf.keras.layers.Average()(X_avg)    
        
    # Linear+ReLu layer
    X = tfl.Dense(units = 128, activation='relu')(X)
    X = tfl.Dropout(rate = 0.4)(X)  

    # Linear+ReLu layer
    X = tfl.Dense(units = 64, activation='relu')(X)
    X = tfl.Dropout(rate = 0.4)(X) 

    # Linear+ReLu layer
    X = tfl.Dense(units = 32, activation='relu')(X)
    X = tfl.Dropout(rate = 0.2)(X) 

    # Linear+Softmax layer
    # Output: y (m, # classes), probability of each class
    X = tfl.Dense(units = n_class, activation='softmax')(X)

    # Model
    model = tf.keras.models.Model(inputs=X_input, outputs=X)

    return model

In [11]:
def model_LSTM(input_shape, word_to_index, word_to_vector=False, n_class=2, trainable=False):
    '''
    input_shape: (max_len,) or (max_len, num_cat)
    word_to_index: word to index dictionary, False for one hot encoding
    word_to_vector: word to embedding vector dictionary, False for one hot encoding

    return model

    then
    X: Indices of a sentence (m, max_len)
    Y: Class probability, one hot vector (m, # classes)
    '''

    # Input layer
    # X_oh (m, max_len, num_cat)
    # X_indices (m, max_len)
    X_input = tfl.Input(shape=input_shape, dtype='int32')

    # Embedding layer
    #embedding_layer = pretrained_embedding_layer(word_to_vector, word_to_index, trainable=trainable)
    #X = embedding_layer(X_indices)   

    # By default, assumes one hot vector input
    # If word_to_index, word_to_vector is provided, add an embedding layer
    X = X_input
    if bool(word_to_vector):
        # Word embeding
        # Output: (m, max_len, emb_dim)
        # Embedding layer
        embedding_layer = pretrained_embedding_layer(word_to_vector, word_to_index, trainable=trainable)
        X = embedding_layer(X)      
        # Masking layer
        # skip zero vector words
        X = tfl.Masking(mask_value=0.)(X)
        
    else:
        X = tf.dtypes.cast(X, tf.float32)

    # LSTM layer
    # Output: a[1] (m, max_len, 128 hidden unit), batch of sequences
    X = tfl.LSTM(units = 128, return_sequences= True)(X)
    X = tfl.Dropout(rate = 0.5 )(X) 

    # LSTM layer
    # Output: a[2]<max_len> (m, 128 hidden unit)
    X = tfl.LSTM(units = 128)(X)
    X = tfl.Dropout(rate = 0.5)(X)  

    # Linear layer
    # Output: a[3] (m, # classes)
    X = tfl.Dense(units = n_class)(X)

    # Softmax layer
    # Output: y (m, # classes), probability of each class
    X = tfl.Activation('softmax')(X)

    # Model
    model = tf.keras.models.Model(inputs=X_input, outputs=X)

    return model

# Model wrapper functions

In [12]:
def run_multiple_NB(model_vars,train,dev,test,Yname='label'):

    results = []

    for var in model_vars:
        
        Xname = var
        
        model_name, y_pred, y_true, proba, classifier, counter = \
            simple_NB(train,dev,test,Xname=Xname,Yname=Yname)
        
        x = test[Xname].to_numpy()
        
        model_name = model_name+' '+Xname
        
        results.append([model_name, x, y_pred, y_true, proba, classifier, counter])
        
    return results

In [13]:
def run_multiple_NN(model_vars, train, dev, test, pos_to_index, word_to_index=False, word_to_vector=False, 
                        Yname='label', max_len=20, n_class=2, 
                        epochs = 20, batch_size = 32, patience=2, trainable=False):
    
    
    results = []

    for i in range(len(model_vars)):
            
        model_name = model_vars[i][0]
        func_model = model_vars[i][1]
        Xname = model_vars[i][2]
        use_embeddings = model_vars[i][3]
        
        model_name = model_name+' '+Xname
        
        w2i = False
        w2v = False
        X_shape = False
        
        if use_embeddings:
            w2i = word_to_index        
            w2v = word_to_vector
            X_shape = (max_len, )
        else:
            w2i = pos_to_index
            X_shape = (max_len, len(pos_to_index))

        _, _, X_train_indices, _, Y_train_oh = dataframe_to_arrays(train, w2i, max_len, Xname=Xname)
        _, _, X_dev_indices,   _, Y_dev_oh   = dataframe_to_arrays(dev, w2i, max_len, Xname=Xname)
        index, _, X_test_indices, Y_test, _  = dataframe_to_arrays(test, w2i, max_len, Xname=Xname)

        
        X = False
        if use_embeddings:
            X = [X_train_indices, X_dev_indices, X_test_indices]
        else:
            X = [to_categorical(X_train_indices, num_classes=len(pos_to_index)), 
                 to_categorical(X_dev_indices, num_classes=len(pos_to_index)), 
                 to_categorical(X_test_indices, num_classes=len(pos_to_index))]            

        
        Y = [Y_train_oh, Y_dev_oh, Y_test]

        
    
        print('should match all: ', X_shape, X[0].shape, X[1].shape, X[2].shape)
    
        model = func_model(X_shape, w2i, w2v, n_class, trainable)
        model.summary()
                    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)  
        
        history = False
        
        if patience :
            history = model.fit(X[0], Y[0], 
                                epochs = epochs, batch_size = batch_size, shuffle=True, 
                                validation_data=(X[1], Y[1]),
                                callbacks=[es])
        else:
            history = model.fit(X[0], Y[0], 
                                epochs = epochs, batch_size = batch_size, shuffle=True, 
                                validation_data=(X[1], Y[1]))
            
        proba = model.predict(X[2])
        y_pred = [np.argmax(proba[i]) for i in range(len(proba))]
        y_true = Y[2]
        x = X[2]
        
        results.append([model_name, x, y_pred, y_true, proba, model, history])
    
    return results

# Run models

In [14]:
seq_type = ['Original','Lower','CleanedWords','PoS','MinimalWords']
ml_type = ['NaiveBayse','FNN','LSTM']
seq_type_abb = ['og','lo','cw','ps','mw']
ml_type_abb = ['nb','fnn','lstm']

n_ml = len(ml_type)
n_seq = len(seq_type)

title_vars = ['org_title','lower_title','cleaned_words','cleaned_pos','minimal_words' ]


nn_vars = []
for i in range(n_seq):
    title = title_vars[i]
    oh = True
    if title=='cleaned_pos':
        oh=False
    nn_vars.append(['FNN',model_FNN,title,oh])
        
for i in range(n_seq):
    title = title_vars[i]
    oh = True
    if title=='cleaned_pos':
        oh=False
    nn_vars.append(['LSTM',model_LSTM,title,oh])
    



In [15]:
# Naive Bayes
res_nb =  run_multiple_NB(title_vars,train,dev,test,Yname='label')
    

# Neural Networks

res_nn = run_multiple_NN(nn_vars, train, dev, test,
                         pos_to_index, word_to_index, word_to_vector, 
                         Yname='label', max_len=max_len, n_class=2,
                         epochs = 50, batch_size = 32, patience=4, trainable=True)

# Add NB and NN
results = res_nb + res_nn

should match all:  (20,) (24000, 20) (8000, 20) (8000, 20)


2022-02-20 03:34:29.642517: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 20, 50)       20000050    ['input_1[0][0]']                
                                                                                                  
 masking (Masking)              (None, 20, 50)       0           ['embedding[0][0]']              
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 50)          0           ['masking[0][0]']                
 ingOpLambda)                                                                                 

750/750 [==============================] - 159s 210ms/step - loss: 0.2982 - accuracy: 0.8751 - val_loss: 0.1865 - val_accuracy: 0.9240
Epoch 2/50
750/750 [==============================] - 158s 210ms/step - loss: 0.1596 - accuracy: 0.9401 - val_loss: 0.1492 - val_accuracy: 0.9426
Epoch 3/50
750/750 [==============================] - 157s 209ms/step - loss: 0.1073 - accuracy: 0.9616 - val_loss: 0.1457 - val_accuracy: 0.9479
Epoch 4/50
750/750 [==============================] - 156s 208ms/step - loss: 0.0750 - accuracy: 0.9732 - val_loss: 0.1381 - val_accuracy: 0.9498
Epoch 5/50
750/750 [==============================] - 155s 207ms/step - loss: 0.0538 - accuracy: 0.9816 - val_loss: 0.1716 - val_accuracy: 0.9463
Epoch 6/50
750/750 [==============================] - 155s 206ms/step - loss: 0.0411 - accuracy: 0.9858 - val_loss: 0.2161 - val_accuracy: 0.9469
Epoch 7/50
750/750 [==============================] - 158s 211ms/step - loss: 0.0315 - accuracy: 0.9898 - val_loss: 0.2069 - val_accura

750/750 [==============================] - 160s 212ms/step - loss: 0.3003 - accuracy: 0.8735 - val_loss: 0.1827 - val_accuracy: 0.9262
Epoch 2/50
750/750 [==============================] - 160s 213ms/step - loss: 0.1589 - accuracy: 0.9417 - val_loss: 0.1503 - val_accuracy: 0.9420
Epoch 3/50
750/750 [==============================] - 159s 212ms/step - loss: 0.1078 - accuracy: 0.9625 - val_loss: 0.1511 - val_accuracy: 0.9476
Epoch 4/50
750/750 [==============================] - 159s 212ms/step - loss: 0.0747 - accuracy: 0.9745 - val_loss: 0.1418 - val_accuracy: 0.9482
Epoch 5/50
750/750 [==============================] - 159s 212ms/step - loss: 0.0546 - accuracy: 0.9817 - val_loss: 0.1640 - val_accuracy: 0.9482
Epoch 6/50
750/750 [==============================] - 156s 209ms/step - loss: 0.0395 - accuracy: 0.9869 - val_loss: 0.1842 - val_accuracy: 0.9475
Epoch 7/50
750/750 [==============================] - 155s 207ms/step - loss: 0.0323 - accuracy: 0.9898 - val_loss: 0.1996 - val_accura

750/750 [==============================] - 158s 210ms/step - loss: 0.2724 - accuracy: 0.8839 - val_loss: 0.1569 - val_accuracy: 0.9390
Epoch 2/50
750/750 [==============================] - 159s 212ms/step - loss: 0.1377 - accuracy: 0.9501 - val_loss: 0.1221 - val_accuracy: 0.9521
Epoch 3/50
750/750 [==============================] - 160s 213ms/step - loss: 0.0908 - accuracy: 0.9684 - val_loss: 0.1116 - val_accuracy: 0.9581
Epoch 4/50
750/750 [==============================] - 157s 209ms/step - loss: 0.0617 - accuracy: 0.9797 - val_loss: 0.1155 - val_accuracy: 0.9581
Epoch 5/50
750/750 [==============================] - 158s 211ms/step - loss: 0.0424 - accuracy: 0.9866 - val_loss: 0.1381 - val_accuracy: 0.9584
Epoch 6/50
750/750 [==============================] - 157s 209ms/step - loss: 0.0341 - accuracy: 0.9893 - val_loss: 0.1534 - val_accuracy: 0.9517
Epoch 7/50
750/750 [==============================] - 157s 210ms/step - loss: 0.0229 - accuracy: 0.9923 - val_loss: 0.1730 - val_accura

Epoch 1/50
750/750 [==============================] - 3s 3ms/step - loss: 0.1552 - accuracy: 0.9486 - val_loss: 0.0874 - val_accuracy: 0.9694
Epoch 2/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0985 - accuracy: 0.9654 - val_loss: 0.0781 - val_accuracy: 0.9710
Epoch 3/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0910 - accuracy: 0.9667 - val_loss: 0.0809 - val_accuracy: 0.9697
Epoch 4/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0847 - accuracy: 0.9688 - val_loss: 0.0765 - val_accuracy: 0.9719
Epoch 5/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0837 - accuracy: 0.9683 - val_loss: 0.0731 - val_accuracy: 0.9736
Epoch 6/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0809 - accuracy: 0.9686 - val_loss: 0.0744 - val_accuracy: 0.9732
Epoch 7/50
750/750 [==============================] - 2s 2ms/step - loss: 0.0796 - accuracy: 0.9699 - val_loss: 0.0745 - val_accuracy: 0.9726
Epoch 

750/750 [==============================] - 162s 214ms/step - loss: 0.3191 - accuracy: 0.8651 - val_loss: 0.1856 - val_accuracy: 0.9240
Epoch 2/50
750/750 [==============================] - 161s 215ms/step - loss: 0.1651 - accuracy: 0.9386 - val_loss: 0.1476 - val_accuracy: 0.9421
Epoch 3/50
750/750 [==============================] - 162s 216ms/step - loss: 0.1129 - accuracy: 0.9592 - val_loss: 0.1385 - val_accuracy: 0.9454
Epoch 4/50
750/750 [==============================] - 161s 215ms/step - loss: 0.0838 - accuracy: 0.9712 - val_loss: 0.1435 - val_accuracy: 0.9473
Epoch 5/50
750/750 [==============================] - 162s 216ms/step - loss: 0.0646 - accuracy: 0.9784 - val_loss: 0.1531 - val_accuracy: 0.9482
Epoch 6/50
750/750 [==============================] - 160s 213ms/step - loss: 0.0499 - accuracy: 0.9829 - val_loss: 0.1816 - val_accuracy: 0.9466
Epoch 7/50
750/750 [==============================] - 158s 211ms/step - loss: 0.0400 - accuracy: 0.9865 - val_loss: 0.1934 - val_accura

750/750 [==============================] - 238s 309ms/step - loss: 0.1966 - accuracy: 0.9203 - val_loss: 0.1238 - val_accuracy: 0.9550
Epoch 2/50
750/750 [==============================] - 229s 306ms/step - loss: 0.0982 - accuracy: 0.9643 - val_loss: 0.1020 - val_accuracy: 0.9611
Epoch 3/50
750/750 [==============================] - 231s 309ms/step - loss: 0.0549 - accuracy: 0.9806 - val_loss: 0.1108 - val_accuracy: 0.9600
Epoch 4/50
750/750 [==============================] - 230s 307ms/step - loss: 0.0325 - accuracy: 0.9889 - val_loss: 0.1317 - val_accuracy: 0.9643
Epoch 5/50
750/750 [==============================] - 225s 300ms/step - loss: 0.0201 - accuracy: 0.9935 - val_loss: 0.1667 - val_accuracy: 0.9591
Epoch 6/50
750/750 [==============================] - 226s 301ms/step - loss: 0.0123 - accuracy: 0.9958 - val_loss: 0.2052 - val_accuracy: 0.9549
Epoch 00006: early stopping
should match all:  (20, 39) (24000, 20, 39) (8000, 20, 39) (8000, 20, 39)
Model: "model_8"
________________

# Print result

In [16]:
def print_result(results):
    
    for result in results:
        
        model_name, x, y_pred, y_true, proba = result[:5]
        
        y_true = y_true.to_numpy()
        
        print(model_name)
        print('accuracy: ',accuracy_score(y_true, y_pred))
        print('precision: ',precision_score(y_true, y_pred))
        print('recall: ',recall_score(y_true, y_pred))
        print('f1: ',f1_score(y_true, y_pred))
        print('\n')

# Organize results into a dataframe

In [17]:
df_result = []

for i in range(len(results)):

    df = pd.DataFrame({
        'x': results[i][1].tolist(),
        'y_true': results[i][3].to_numpy().tolist(),
        'y_pred': results[i][2],
        'proba0': [results[i][4][j][0] for j in range(len(results[i][1])) ],
        'proba1': [results[i][4][j][1] for j in range(len(results[i][1])) ]
    }, index = results[i][3].index)
    df_result.append(df)

In [18]:
ml_type = ['NaiveBayse','FNN','LSTM']
seq_type_abb = ['og','lo','cw','ps','mw']
ml_type_abb = ['nb','fnn','lstm']

n_ml = len(ml_type)
n_seq = len(seq_type)

title_vars = ['org_title','lower_title','cleaned_words','cleaned_pos','minimal_words']

dict_name = []

for i in range(n_ml):
    for j in range(n_seq):
        
        y_pred = ml_type_abb[i]+'_'+seq_type_abb[j]
        proba0 = y_pred+'_p0'
        proba1 = y_pred+'_p1'
        temp_dict = {'y_pred':y_pred,'proba0':proba0,'proba1':proba1}
        
        if i==0:
            x = title_vars[j]
            temp_dict['x'] = x
        
        dict_name.append(temp_dict)

df = []

df.append(df_result[0].rename(columns = dict_name[0]))
for i in range(1,n_seq):
    df.append(df_result[i].drop(['y_true'], axis=1, inplace=False).rename(columns = dict_name[i]))

for i in range(n_seq, n_ml*n_seq):
    df.append(df_result[i].drop(['x','y_true'], axis=1, inplace=False).rename(columns = dict_name[i]))


In [19]:
df_x = df[0][title_vars[0]].to_frame()
for i in range(1,n_seq):
    df_x = pd.merge(df_x, df[i][title_vars[i]].to_frame(), left_index=True, right_index=True)
    #print(i,len(df_x))

df_y = df[0].y_true.to_frame() 
print('y',len(df_y))
    
df[0].drop(['y_true'], axis=1, inplace=True)
for i in range(0,n_seq):
    df[i].drop([title_vars[i]], axis=1, inplace=True)

df_p = df[0]    
for i in range(1,n_seq*n_ml):
    df_p = pd.merge(df_p,df[i], left_index=True, right_index=True)
    
df_ml = pd.merge(df_x, df_y, left_index=True, right_index=True)
df_ml = pd.merge(df_ml, df_p, left_index=True, right_index=True)

y 8000


In [20]:
cut_name, y_true_cut, y_pred_cut = simple_Manual(test)

In [21]:
dict_manual = {cut_name[i]:y_pred_cut[i] for i in range(len(cut_name))}

df_manual = pd.DataFrame(dict_manual)

#display(df_manual)

In [22]:
df_final = pd.merge(df_ml, df_manual, left_index=True, right_index=True)


In [23]:
#display(df_ml)
#display(df_final)
display(df_final.sample(10))

print(df_final.columns)

df_final.to_csv('data/model_compare2.csv',index=False)

,org_title,lower_title,cleaned_words,cleaned_pos,minimal_words,y_true,nb_og,nb_og_p0,nb_og_p1,nb_lo,...,lstm_mw_p0,lstm_mw_p1,too_long,noisy,clickbait,slang,donald,obama,hillary,bernie
35973,"BREAKING: OBAMA’S “Director Of Diversity” Donates Hundreds Of Thousands Of Tax Payer Dollars To Open Borders, Other Radical Groups","breaking: obama’s “director of diversity” donates hundreds of thousands of tax payer dollars to open borders, other radical groups",breaking : obama s director of diversity donates hundreds of thousands of tax payer dollars to open borders other radical groups,NN : NNP NN IN NNP NNP NNP IN NNP IN NNP NNP NNP TO VB NNP JJ JJ NNS,breaking obama director diversity donates hundred thousand tax payer dollar open border radical group,1,1,5.057249e-07,0.999999,1,...,0.000005,0.999995,0,0,0,0,0,1,0,0
12307,Senate Democratic leader urges removal of House intel panel chair,senate democratic leader urges removal of house intel panel chair,senate democratic leader urges removal of house intel panel chair,NNP JJ NN NNS NN IN NNP NN NN NN,senate democratic leader urge removal house intel panel chair,0,0,9.999941e-01,0.000006,0,...,0.999940,0.000060,0,0,0,0,0,0,0,0
20886,"BREAKING: [Video] DIRTY BOMB FEARS, As NYC Police Officers With Targets Placed On Their Backs By Obama And DeBlasio Wear Nuclear Bomb Detectors July 4th Weekend","breaking: [video] dirty bomb fears, as nyc police officers with targets placed on their backs by obama and deblasio wear nuclear bomb detectors july 4th weekend",breaking : _mytag_parentheses_ dirty bomb fears as nyc police officers with targets placed on their backs by obama and deblasio wear nuclear bomb detectors july 4th weekend,NN : JJ NNP NNP NNP IN NNP NNP NNP IN NNP NNP IN NNP NNP IN NNP CC NNP NNP NNP NNP NNP NNP CD NNP,breaking _mytag_parentheses_ dirty bomb fear nyc police officer target placed back obama deblasio wear nuclear bomb detector july 4th weekend,1,1,2.207801e-08,1.000000,1,...,0.000004,0.999996,1,0,1,0,0,1,0,0
16217,U.S. condemns Venezuelan elections as neither free nor fair,u.s. condemns venezuelan elections as neither free nor fair,u . s . condemns venezuelan elections as neither free nor fair,NNP NN NNP NNS IN DT JJ CC JJ,condemns venezuelan election neither free fair,0,0,9.965800e-01,0.003420,0,...,0.858894,0.141106,0,0,0,0,0,0,0,0
26943,HOOKERS FOR HILLARY: Why They’ve Got Her Back…[VIDEO],hookers for hillary: why they’ve got her back…[video],hookers for hillary : why they ve got her back _mytag_parentheses_,NNS NNP NNP : WRB NNP NNP NNP NNP NN,hooker hillary got back _mytag_parentheses_,1,1,4.339067e-11,1.000000,1,...,0.000236,0.999764,0,0,1,0,0,0,1,0
7374,Cambodia revokes diplomatic passports of banned opposition members,cambodia revokes diplomatic passports of banned opposition members,cambodia revokes diplomatic passports of banned opposition members,NNP VBZ JJ NNS IN JJ NN NNS,cambodia revoke diplomatic passport banned opposition member,0,0,9.999564e-01,0.000044,0,...,0.999992,0.000008,0,0,0,0,0,0,0,0
30715,"WHY THIS BLUE-COLLAR DEMOCRAT STRONGHOLD County Is Still Fighting For Trump: “He was the hope we were all waiting on, the guy riding up on the white horse” [VIDEO]","why this blue-collar democrat stronghold county is still fighting for trump: “he was the hope we were all waiting on, the guy riding up on the white horse” [video]",why this blue collar democrat stronghold county is still fighting for trump : he was the hope we were all waiting on the guy riding up on the white horse _mytag_parentheses_,WRB NNP NNP NNP NNP NNP NNP RB VBG IN NNP : NN VBD DT NN PRP VBD DT VBG IN DT NN VBG RP IN DT JJ NN NN,blue collar democrat stronghold county still fight trump hope wait guy rid white horse _mytag_parentheses_,1,1,6.310488e-12,1.000000,1,...,0.000593,0.999407,1,1,1,0,0,0,0,0
26191,REAL-TIME DEBATE Graph Shows Trump CRUSHED Hillary With Independent Voters…A MUST WATCH!,real-time debate graph shows trump crushed hillary with independent voters…a must watch!,real tim

Index(['org_title', 'lower_title', 'cleaned_words', 'cleaned_pos',
       'minimal_words', 'y_true', 'nb_og', 'nb_og_p0', 'nb_og_p1', 'nb_lo',
       'nb_lo_p0', 'nb_lo_p1', 'nb_cw', 'nb_cw_p0', 'nb_cw_p1', 'nb_ps',
       'nb_ps_p0', 'nb_ps_p1', 'nb_mw', 'nb_mw_p0', 'nb_mw_p1', 'fnn_og',
       'fnn_og_p0', 'fnn_og_p1', 'fnn_lo', 'fnn_lo_p0', 'fnn_lo_p1', 'fnn_cw',
       'fnn_cw_p0', 'fnn_cw_p1', 'fnn_ps', 'fnn_ps_p0', 'fnn_ps_p1', 'fnn_mw',
       'fnn_mw_p0', 'fnn_mw_p1', 'lstm_og', 'lstm_og_p0', 'lstm_og_p1',
       'lstm_lo', 'lstm_lo_p0', 'lstm_lo_p1', 'lstm_cw', 'lstm_cw_p0',
       'lstm_cw_p1', 'lstm_ps', 'lstm_ps_p0', 'lstm_ps_p1', 'lstm_mw',
       'lstm_mw_p0', 'lstm_mw_p1', 'too_long', 'noisy', 'clickbait', 'slang',
       'donald', 'obama', 'hillary', 'bernie'],
      dtype='object')
